In [ ]:
import requests
import hashlib

ip_local="https://127.0.0.1"
#ip_local="https://192.168.1.185"
ip_remote="https://101.43.239.148"
ip=ip_remote
#在本地跑服务器的话，ip=ip_local
self_sign_rootca = 'cert/server/rootCA.pem'
log_payload = {
    "name": "root",
    "password": hashlib.md5(b'root').hexdigest(),
}
reg_payload = {
    "name": "root",
    "password": "root",
}
#hello world
response = requests.get(ip,verify=self_sign_rootca)
print(response.text)

In [ ]:
#register
response = requests.post(ip+'/api/register',data=reg_payload,verify=self_sign_rootca)
print(response.text)

In [ ]:
#login session
session = requests.Session()
response =session.post(ip+'/api/login',data=log_payload,verify=self_sign_rootca)
print(log_payload)
print(response.text)

In [ ]:
#bind device
#登录后还要绑定设备才能进行operate操作，否则/api/operate会返回{
#   "code": 400, 
#   "msg": "Please bind device first"
# }

response =session.post(ip+'/api/bindDevice',data={"pin":"123456"},verify=self_sign_rootca)
print(response.text)

In [ ]:
#operate(payload):
operate_payload = {
    "R": "1",
    "Param": "0",
}
# 开灯  1 0
# 关灯  1 1
# 调亮  3 0
# 调暗  3 1   
response = session.post(ip+'/api/operate', data=operate_payload,verify=self_sign_rootca)
print(response.text)

In [ ]:
#getstatus(payload):
response = session.post(ip+'/api/getstatus',verify=self_sign_rootca)
print(response.text)

In [ ]:
#channel(payload):
channel_get = {
    'operate':'get',
    }
channel_set = {
    'operate':'set',
    'target':'speaker',
    'param':'on',
    }
response = session.post(ip+'/api/channel',data=channel_get,verify=self_sign_rootca)
print(response.text)

In [ ]:
#getMessages:
#返回用户所有未处理已处理的申请信息  像是这样  apply_id是供服务器间同步的
# {
#   "code": 200, 
#   "msg": [
#     {
#       "R": "1", 
#       "apply_id": 1, 
#       "channel": "zigbee", 
#       "id": 1, 
#       "name": "test", 
#       "status": "toprocess"
#     }
#   ]
# }

response = session.post(ip+'/api/getMessages',verify=self_sign_rootca)
print(response.text)

In [ ]:
#模拟zigbee服务器向manu发送token请求:
#现在要添加token需要调用这个   调用后用上面的getmessage就能看到申请存在数据库里   需要用户主动调用processMessages处理

#测试的话就给自己(主人用户)申请token
token_apply = {
    "name": "test",
    "R": "0",
}
response = session.post(ip+'/api/applyToken',verify=self_sign_rootca,data=token_apply)
print(response.text)

In [ ]:
#processMessages:
process_payload={
    "apply_id":"2",
    "operate":"accept",#或者reject
    }
response = session.post(ip+'/api/processMessages', data=process_payload,verify=self_sign_rootca)
print(response.text)

In [ ]:
#delToken:
#主人用户主动删除权限
delToken_payload={
    "name":"test",
    "R": "1", 
    }
response = session.post(ip+'/api/delToken', data=delToken_payload,verify=self_sign_rootca)
print(response.text)

In [ ]:
#getDeviceUser
response=session.post(ip+'/api/getDeviceUser',verify=self_sign_rootca)
print(response.text)

In [ ]:
#getLogs:
#返回服务器日志

response = session.post(ip+'/api/getLogs',verify=self_sign_rootca)
print(response.text)

In [ ]:
#调用后清空数据库
response =requests.post(ip+'/dbreset',verify=self_sign_rootca)
print(response.text)